In [4]:
%%javascript
// Making sure the outputs display correctly
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [5]:
# Importing required packages for this homework
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
#from IPython.display import HTML, display
#import tabulate
%matplotlib inline

def rounds(item):
    ret = float('%.2f'% (item * 100/100))
    return(ret)

path = os.getcwd()
print(path)

C:\Users\butle175\Downloads\rocketProp-master


In [6]:
# https://www.grc.nasa.gov/WWW/k-12/airplane/atmosmet.html
class atmosphere:
    def __init__(self, val, valGiven = 0,units = "SI"):
        #Convert from US to SI
        if units != "SI" and valGiven == 0:
            val = val / 3.281
        if units != "SI" and valGiven == 1:
            val = val * 0.04788
        #0 implies the given value is an altitude
        #1 implies the given value is a pressure
        if valGiven == 0:
            self.h = val
        elif valGiven == 1:
            self.P = val
        else:
            print("Not a valid 'valGiven' parameter.")
    def hCalc(self):
        if self.h < 11000:
            self.T = 15.04 - 0.00649*self.h #K
            self.P = 101.29 * ((self.T + 273.1)/288.08)**(5.256) #kPa
        elif self.h < 25000:
            self.T = -56.46
            self.P = 22.65 * np.exp(1.73 - 0.000157*self.h)
        elif self.h > 24999:
            self.T = -131.21 + 0.00299*self.h
            self.P = 2.488 * ((self.T + 273.1)/216.6)**(-11.388)
        self.rho = self.P / (0.2869 * (self.T + 273.1))
    def PCalc(self):
        if self.P > 22.632:
            self.T = (288.08*(self.P/101.29)**(1/5.256))-273.1
            self.h = (self.T - 15.04)/(-0.00649)
        elif self.P > 0.1113586:
            self.T = -56.46
            self.h = (1.73 - np.log((self.P/22.65)))/(0.000157)
        else:
            self.T = (216.6*(self.P/2.488)**(1/(-11.388)))-273.1
            self.h = (self.T + 131.21)/0.00299
        self.rho = self.P / (0.2869 * (self.T + 273.1))

In [7]:
#Finding angle it is falling at
angRad = 195 #pixels
angX = 124 #pixels
theta = np.arccos(124/195)
#print(rounds(theta),'radians')
#From flat going down
thetaFlat = np.pi - theta - np.pi/2
#print(rounds(thetaFlat))

Afin = (17.537 * 0.0254 ** 2)*np.cos(thetaFlat) #sq in

radRock = (8.5 / (np.pi * 2)) * 0.0254 #m
print("diameter:",(radRock*2)) #meters
lenNose = 9 * 0.0254 #m
lenBody = 30 * 0.0254 #m
Aflat = (radRock**2 * np.pi)+(radRock*2*lenBody)
Abody = Aflat * np.cos(thetaFlat) #sq in

totAreaFalling = Abody + Afin*2
print('falling area (US):', totAreaFalling)
print('Falling area:',totAreaFalling,'sq meters')
print((11*0.0254)**2*np.pi)

diameter: 0.0687231044270804
falling area (US): 0.06074218991561928
Falling area: 0.06074218991561928 sq meters
0.24524641988318885


In [8]:
## Cd with parachute
CdPara = (2 * 4.44822) / (1.229 * (0.06074 + ((11*0.0254)**2*np.pi)) * (1.8288**2))
print(CdPara)

7.073425647864539


In [48]:
#Cubic regression
def cubReg(points):
    n = len(points)
    X = [item[0] for item in points]
    Y = [item[1] for item in points]
    Sx = sum(X)
    Sy = sum(Y)
    S2x = sum([item**2 for item in X])
    S3x = sum([item**3 for item in X])
    S4x = sum([item**4 for item in X])
    S5x = sum([item**5 for item in X])
    S6x = sum([item**6 for item in X])
    Sxy = sum([X[i] * Y[i] for i in range(n)])
    S2xy = sum([X[i]**2 * Y[i] for i in range(n)])
    S3xy = sum([X[i]**3 * Y[i] for i in range(n)])
    A = [[S6x,S5x,S4x,S3x],[S5x,S4x,S3x,S2x],[S4x,S3x,S2x,Sx],[S3x,S2x,Sx,n]]
    b = [[S3xy],[S2xy],[Sxy],[Sy]]
    coef = np.linalg.solve(A, b)
    return(coef)
#Logarithmic Regression
def logReg(points):
    n = len(points)
    X = [item[0] for item in points]
    Y = [item[1] for item in points]
    Sx = sum([np.log(item) for item in X])
    Sy = sum(item for item in Y)
    S2x = Sx**2
    nSx2 = n*sum([(np.log(item))**2 for item in X])
    nSyx = n * sum([Y[i]*np.log(X[i]) for i in range(n)])
    SySx = Sx * Sy
    b = (
        (nSyx - SySx)/
        (nSx2 - S2x)
    )
    a = ( 
        (Sy - b*Sx)/
        n
    )
    return([a,b])
    
#Reynolds numbers for alt=0 in meters
Rey0 = [[0,7900],[22,7200],
        [40,6750],[60,6200],
        [80,5800],[100,5400],
       [38,6800],[54,6400]]
alt0 = cubReg(Rey0)
print("Regression coefficients:",alt0)

#test figure
Read = alt0[0][0]*32**2 + alt0[1][0]*32 + alt0[2][0]
print("Graph reading:",Read)
print("Reynolds:",Read*(39/12)*100)

Regression coefficients: [[ 6.76331346e-02]
 [-3.16725650e+01]
 [ 7.89256250e+03]]
Graph reading: 6948.296750032716
Reynolds: 2258196.4437606325


In [139]:
#Coefficient of friction values (Turbulant)
CfT = [[130000,0.006],[1650000,0.004],
      [160000,.0058],[320000,0.0052],
      [2700000,0.0037],[7000000,0.0032]]
CfTReg = logReg(CfT)

#Coefficient of friction values (Laminar)
CfL = [[50000,0.006],[440000,0.002],
       [900000,0.0014],[700000,0.0016],
      [200000,0.003],
      [92000,0.0044],
      [1800000,0.001],[2200000,0.00092],
      [1050000,0.0013],[1450000,0.0011],
      [1900000,0.00096],[44000,0.0064],
      [86000,0.0046],[1200000,0.0012],
      [1600000,0.00105],[1750000,.001],
      [975000,0.0014]]

CfLReg = logReg(CfL)

print(0.664/np.sqrt(200000))
print(0.027/(1000000)**(1/7))

0.0014847491370598605
0.0037516378348074722


In [108]:
#Wet surface area to Body surface area
SwSbtNose = 8.8
SwSbtBody = 44.4
SwSbt = SwSbtNose + SwSbtBody
print(SwSbt)

53.2


In [140]:
#####################################
##       START OF CODE FILE        ##
#####################################

##### DON'T CHANGE THESE #####
#Wet surface area to body surface area
SwSbt = 52.6
# Length over radius
Ld = 39/2.705634
# Length of the rocket
L = 39/12 #feet
# Diameter of rocket
D = 2.705634 #inches

##### CHANGE THIS ON DAY #####
T = 32 #Farenheit

##### REGRESSIONS AND FUNCTIONS #####
#Quadratic Regression for Determining Reynolds Number
## Temperatures are in Kelvin
## Re = aT^2 + bT + c
a = alt0[0][0]
b = alt0[1][0]
c = alt0[2][0]
#Find the unit value of Reynolds from temprature in kelvin
def reynolds(T,vel, L = (39/12)):
    ReUnit = a*T**2 + b*T + c
    #print('Reading:',ReUnit)
    Re = ReUnit * vel * L
    return(Re)

#Linear regression of turbulant coefficient 
## of friction from Reynolds.
## Take values directly from above
## Cf = m(Re) + b
def CfTurbSolve(Re):
    #From the regression above
    Cf = CfTReg[0] + CfTReg[1]*np.log(Re)
    return(Cf)

#Linear regression of  Laminar coefficient 
## of friction from Reynolds.
## Take values directly from above
## Cf = m(Re) + b
def CfLamSolve(Re):
    #From the regressions above
    Cf = CfLReg[0] + CfLReg[1]*np.log(Re)
    return(Cf)

#Velo = 100 * 3.28
#test = reynolds(280,Velo)
#print(test)
#del(test)

# Calculating Cd from these values
def Cd(T, vel, Laminar=True,L=(39/12),):
    Rey = reynolds(T,vel,L)
    print("rey:",Rey)
    if Laminar:
        #Using Blasius Solution
        Cf = 0.664/np.sqrt(Rey)
    else:
        #Using Prandtl's one-seventh power law
        Cf = 0.027/(Rey**(1/7))
    Cd = 1.02 * Cf * (1 + 
                      (1.5 / (Ld)**(3/2))
                     ) * SwSbt
    return(Cd)

#v = np.linspace(0,300,10)
#for item in v:
#    print("Cd for ",item," ft/s \n",Cd(32,item))

'''
You can build a for loop around the Cd function and 
from there determine the coefficient of drag for every speed
you might find along the trajectory
'''

#Example case
v = 400
testCase = Cd(T,v,Laminar=False)
print("Coefficient of drag for " + str(T)+"F and "
      + str(v) + " ft/s:\n",testCase)




rey: 9032785.77504253
Coefficient of drag for 32F and 400 ft/s:
 0.1510095301285275


In [136]:
#Drag for the fins
R_fin = reynolds(32,400,(6/12))
#print(R_fin)
tipChord = 0.125/4.5
CfFin = CfLamSolve(R_fin)
#print("Cf:",CfFin)
CDStar = 2 * CfFin*(1 + 2*tipChord)
Sf = ((6*4.5*0.5) + (4.5*3*0.5))*3 #sq in
Sbt = 1.3528**2 * np.pi #sq in
print(Sf,Sbt)
CD0 = CDStar * (Sf/Sbt)
print("Coef* Fin:",CDStar)
print("Coef Fin:",CD0)

#Interface Drag
CDint = CDStar * ((6*D)/(2 * Sbt)) * 3
print("Coef Int:",CDint)


60.75 5.7493276817149415
Coef* Fin: 0.0022691559124973977
Coef Fin: 0.02397692901078442
Coef Int: 0.009610784347728102


In [117]:
#Base drag
## This takes in the body drag as a parameter
def baseDrag(bodyDrag):
    Cdb = 0.029/np.sqrt(bodyDrag)
    return(Cdb)
baseDrag(0.15949345)

0.07261503845998567

In [127]:
def llDrag(lugLength,lugDiam):
    Sll = (lugDiam/2)**2 * np.pi * lugLength
    Sllw = Sll + ((lugDiam-0.0625)/2)**2 * np.pi * lugLength
    print(Sllw)
    CDLL = (1.2 * Sll + 0.0045*Sllw)/Sbt
    return(CDLL)
print(llDrag(6,0.5))

2.0800779483729293
0.07616018501721855


In [141]:
testCase + CD0 + CDint + 0.0726 + 0.0762

0.33339724348704